In [2]:
import re
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDFS, RDF, OWL
import rdflib.util
from owlready2 import get_ontology, OwlReadyOntologyParsingError, Thing

def normalize_name(name):
    """
    Normalize a name by converting to lowercase and removing separators.

    Args:
    - name (str): The name to normalize.

    Returns:
    - str: The normalized name.
    """
    return re.sub(r'[\s\-_]+', '', name.lower())

In [3]:
def get_local_name(uri):
    """
    Extract the local name from a URI.

    Args:
    - uri (URIRef): The URI to extract the local name from.

    Returns:
    - str: The local name.
    """
    if isinstance(uri, URIRef):
        return uri.split('/')[-1].split('#')[-1]
    return str(uri)

def load_ontology_files(file_paths):
    """
    Load ontology files into RDF graphs or OWL ontologies.

    Args:
    - file_paths (dict): A dictionary where keys are the names and values are the file paths of the ontology files.
    
    Returns:
    - dict: A dictionary where keys are the names and values are the loaded RDF graphs or OWL ontologies.
    """
    graphs = {}
    for name, file_path in file_paths.items():
        file_extension = file_path.split('.')[-1].lower()
        if file_extension == 'owl':
            try:
                # Try to load OWL file using owlready2
                onto = get_ontology(file_path).load()
                graphs[name] = onto
            except OwlReadyOntologyParsingError:
                # Fallback to rdflib if owlready2 fails
                g = Graph()
                g.parse(file_path, format='xml')
                graphs[name] = g
        else:
            # Load other RDF files using rdflib
            g = Graph()
            if file_extension == 'xrdf':
                g.parse(file_path, format='xml')
            elif file_extension == 'ttl':
                g.parse(file_path, format='turtle')
            else:
                g.parse(file_path, format=rdflib.util.guess_format(file_path))
            graphs[name] = g
    return graphs